# Construir banco de dados vetorial
Obtêm dados do JSON de produtos e efetua o push pro database

## Preparar ambiente

In [ ]:
# Instalar dependências
#!pip install pinecone

In [4]:
from pinecone import Pinecone, ServerlessSpec
import openai
import pandas as pd
import os
import dotenv

### Variáveis de ambiente

In [10]:
dotenv.load_dotenv()

True

In [14]:
token = os.getenv("OPENROUTER_API_KEY") # token do OpenRouter
model_name = os.getenv("MODEL_NAME") # nome do modelo

pinecone_api_key = os.getenv("PINECODE_API_KEY") # token do Pinecone
pinecode_index_name = os.getenv("PINECODE_INDEX_NAME")  # nome do índice do Pinecone

### Iniciar Pinecode

In [15]:
pc = Pinecone(api_key=pinecone_api_key)

# client = OpenAI(api_key=token, base_url=open_ai_base_url)